## 使用 Keras 和 Tensorflow Hub 对电影评论进行文本分类

In [2]:
import sys
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf 
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import numpy as np


%matplotlib inline

print("python version: ", sys.version_info)
for module in np, mpl, tf, keras, hub, tfds:
    print(module.__name__, "version: ", module.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "availabel" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

seed = 10383

python version:  sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
numpy version:  1.18.5
matplotlib version:  3.2.2
tensorflow version:  2.3.0
tensorflow.keras version:  2.4.0
tensorflow_hub version:  0.9.0
tensorflow_datasets version:  2.1.0
Eager mode:  True
GPU is availabel


### 1. 下载 IMDB 数据集

In [3]:
# 将训练集分割成 60% 和 40%，从而最终我们将得到 15,000 个训练样本
# 10,000 个验证样本以及 25,000 个测试样本。
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True, # True: (input, label) False: dict
)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteU77GKO/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteU77GKO/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteU77GKO/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


### 2. 探索数据

In [5]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [6]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

### 3. 构建模型

神经网络由堆叠的层来构建，这需要从三个主要方面来进行体系结构决策：  
1. 如何表示文本？  
2. 模型里有多少层？  
3. 每个层里有多少隐层单元( hidden units ) ？  
本示例中，输入数据由句子组成。预测的标签为 0 或 1。  
  
表示文本的一种方式是将句子转换为嵌入向量（embeddings vectors）。我们可以使用一个预先训练好的文本嵌入（text embedding）作为首层，这将具有三个优点：  
1. 不必担心文本预处理
2. 可以从迁移学习中受益  
3. 嵌入具有固定长度，更易于处理  
  
针对此示例我们将使用 TensorFlow Hub 中名为 <a href="https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1">google/tf2-preview/gnews-swivel-20dim/1</a> 的一种 **预训练文本嵌入（text embedding）模型** 。
  
为了达到本教程的目的还有其他三种预训练模型可供测试:   
* <a href="https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1">google/tf2-preview/gnews-swivel-20dim-with-oov/1</a> ——类似 google/tf2-preview/gnews-swivel-20dim/1，但 2.5%的词汇转换为未登录词桶（OOV buckets）。如果任务的词汇与模型的词汇没有完全重叠，这将会有所帮助。  
* <a href="https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1">google/tf2-preview/nnlm-en-dim50/1</a> ——一个拥有约 1M 词汇量且维度为 50 的更大的模型。  
* <a href="https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1">google/tf2-preview/nnlm-en-dim128/1</a> ——拥有约 1M 词汇量且维度为128的更大的模型。  
  
让我们首先创建一个使用 Tensorflow Hub 模型嵌入（embed）语句的Keras层，并在几个输入样本中进行尝试。请注意无论输入文本的长度如何，嵌入（embeddings）输出的形状都是：(num_examples, embedding_dimension)。  


In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [8]:
model = keras.Sequential()
model.add(hub_layer)
model.add(keras.layers.Dense(16, activation="relu"))
model.add(keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


#### 3.1 损失函数与优化器

In [9]:
model.compile(
    optimizer="adam", 
    loss=keras.losses.BinaryCrossentropy(from_logits=True), 
    metrics=["accuracy"],
)

### 4. 训练模型

In [10]:
history = model.fit(train_data.shuffle(10000).batch(512), 
                    epochs=20, 
                    validation_data=validation_data.batch(512), 
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 3s 95ms/step - loss: 0.6556 - accuracy: 0.6145 - val_loss: 0.6222 - val_accuracy: 0.6189
Epoch 2/20
30/30 [==============================] - 3s 94ms/step - loss: 0.6012 - accuracy: 0.6514 - val_loss: 0.5902 - val_accuracy: 0.6487
Epoch 3/20
30/30 [==============================] - 3s 92ms/step - loss: 0.5674 - accuracy: 0.6812 - val_loss: 0.5596 - val_accuracy: 0.6895
Epoch 4/20
30/30 [==============================] - 3s 93ms/step - loss: 0.5341 - accuracy: 0.7127 - val_loss: 0.5297 - val_accuracy: 0.7121
Epoch 5/20
30/30 [==============================] - 3s 95ms/step - loss: 0.4996 - accuracy: 0.7414 - val_loss: 0.4993 - val_accuracy: 0.7466
Epoch 6/20
30/30 [==============================] - 3s 92ms/step - loss: 0.4636 - accuracy: 0.7703 - val_loss: 0.4695 - val_accuracy: 0.7635
Epoch 7/20
30/30 [==============================] - 3s 92ms/step - loss: 0.4295 - accuracy: 0.7953 - val_loss: 0.4430 - val_accuracy: 0.7767
Epoch 8/20
30

### 5. 评估模型

In [11]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

49/49 - 3s - loss: 0.3270 - accuracy: 0.8571
loss: 0.327
accuracy: 0.857
